In [1]:
!pip install --upgrade pip
!pip install geopandas
!pip install selenium
!pip install webdriver_manager
!pip install plotly
!pip install gif

Antes de desenvolver o grafico vou rodar um script Selenium para baixar a versão atual dos dados do site do ministerio da saude

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(ChromeDriverManager().install(),options=options)

C:\Users\pheli\AppData\Local\Temp/ipykernel_31904/674006772.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(),options=options)


In [3]:
driver.get('https://covid.saude.gov.br/')
driver.find_element(By.XPATH,'/html/body/app-root/ion-app/ion-router-outlet/app-home/ion-content/div[1]/div[2]/ion-button').click()

Agora vamos preparar os dados para o grafico

In [52]:
import pandas as pd
import os
import plotly.express as px
import plotly.graph_objects as go
import gif
from urllib.request import urlopen
import json
PATH = 'csv/'

In [31]:
files = [f for f in os.listdir(PATH) if os.path.isfile(os.path.join(PATH, f))]
df = pd.DataFrame()
for f in files:
  df = pd.concat([df,pd.read_csv(PATH + f, encoding = "ISO-8859-1", sep = ';')])
print(df.shape)

(4805113, 17)
(4762350, 17)


,regiao,estado,municipio,coduf,codmun,codRegiaoSaude,nomeRegiaoSaude,data,semanaEpi,populacaoTCU2019,casosAcumulado,casosNovos,obitosAcumulado,obitosNovos,Recuperadosnovos,emAcompanhamentoNovos,interior/metropolitana
5251,Norte,RO,Alta Floresta D'Oeste,11,110001.0,11005.0,ZONA DA MATA,2022-01-03,1,22945.0,4685.0,5,73,1,NaN,NaN,0.0
246344,Nordeste,PB,Guarabira,25,250630.0,25002.0,2Âª REGIAO,2022-01-04,1,58833.0,10264.0,0,151,0,NaN,NaN,0.0


In [32]:
df.dropna(subset=['municipio'],inplace=True)
df = df.sort_values(by=['semanaEpi'])
df = df[['estado','semanaEpi','casosAcumulado','obitosAcumulado','populacaoTCU2019']].groupby(['estado','semanaEpi']).sum().reset_index()
df['ppCasos'] = df["casosAcumulado"].div(df["populacaoTCU2019"].values)
df['ppObitos'] = df["obitosAcumulado"].div(df["populacaoTCU2019"].values)
print(df.shape)
df.head(2)

(1431, 7)


,estado,semanaEpi,casosAcumulado,obitosAcumulado,populacaoTCU2019,ppCasos,ppObitos
0,AC,1,915691.0,18659,12347090.0,0.074162,0.001511
1,AC,2,929227.0,18783,12347090.0,0.075259,0.001521


In [18]:
with urlopen("https://raw.githubusercontent.com/codeforamerica/click_that_hood/master/public/data/brazil-states.geojson") as response:
 Brazil = json.load(response) # Javascrip object notation 

for feature in Brazil ['features']:
 feature['id'] = feature['properties']['sigla']

Agora ao grafico!

In [ ]:
fig = px.choropleth(df.loc[df['semanaEpi']<=52], 
                    locations ="estado", 
                    color ="ppCasos", 
                    color_continuous_scale = [[0, 'rgb(255,255,255)'],[1, 'rgb(255,0,0)']],
                    range_color = [0,0.15],
                    geojson = Brazil,
                    hover_name ="estado",
                    animation_frame ="semanaEpi") 

fig.update_geos(fitbounds = "locations", visible = False)
fig.show()

Agora vou exportar um gif com esse grafico animado

In [85]:
@gif.frame
def plot(i):
    d = df[df['semanaEpi'] == i]
    fig = go.Figure()
    fig.add_trace(trace=go.Choropleth(locations = df["estado"], 
                    z = df["ppCasos"], 
                    colorscale = 'YlGn',
                    zmin = 0,
                    zmax = 0.15,
                    geojson = Brazil,
                    showlegend=False,
                    showscale=False))
    fig.update_layout(width=500, height=300)
    fig.update_geos(fitbounds = "locations", visible = False)
    return fig

In [83]:
frames = []
for i in range(52):
    frame = plot(i)
    frames.append(frame)

gif.save(frames, 'example.gif', duration=100)

In [86]:
plot(1).show()